In [ ]:
#whole lotta inputs ASTRO RELated
from astroquery.vizier import Vizier
import astropy.units as u
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import astropy.coordinates as coord
import astropy.units as u
import csv
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.wcs import WCS
import astroquery
from astropy.coordinates import SkyCoord
from astroquery.mast import Observations
from astroquery.mast import Catalogs
from astroquery.mast import Tesscut
import astropy.io.ascii as ascii
from astropy.table import Table, join, unique
import pandas as pd



import os
import time
import sys
import shutil
from datetime import datetime
import math
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patheffects as PathEffects
import matplotlib.colors as pltcolors
from matplotlib import gridspec
import matplotlib.ticker as ticker
import matplotlib.colors as csx
import colorsys
import pandas as pd
import numpy as np
import colormaps as cmaps
from scipy.stats import norm

from astropy.stats import mad_std
from astropy.io import fits
from astropy.timeseries import LombScargle
from astropy.coordinates import ICRS, SkyCoord, Galactic, Galactocentric, GalacticLSR, \
                    LSR, CartesianRepresentation
import astropy.units as u
import astropy.constants as c
from astropy.table import Table

from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import astropy
import matplotlib
from astropy.visualization.wcsaxes import WCSAxes

from collections import Counter
import warnings

#machine learning inputs
